In [1]:
# 이걸 잠시 하다보니 다른 생각이 든다..
# 필터링을 위한 메타데이터..가 존재해야하나? 벡터 db에서 필터링을 해야하나?
# 단순히 연산이 작아보이고 쉬워보여서.. 대충 메타데이터에 넣고, 벡터db에서 필터링 하게 한 게 잘못된 건 아닐까?
# 그냥 이러한 필터링 자체는 최대한 rdb 단에서 수행하고, 메타데이터와 벡터db의 사용을 최적화 하는 게 더 좋지 않을까?
# => 그렇게 하자. 처음부터 다시하자.

In [9]:
def populate_code_tables(cursor):
    """
    엑셀 파일의 각 시트를 읽어 코드 테이블에 데이터를 삽입합니다.
    """
    print("🚀 코드 테이블 데이터 이관 시작...")

    try:
        # 테이블과 시트 이름, 그리고 해당 시트 내의 컬럼명을 매핑합니다.
        # 시트 이름은 실제 엑셀 파일의 시트 이름과 정확히 일치해야 합니다.
        code_sheets_map = {
            'major_codes': ('코드정보', 'plcyMajorCd', '코드', '코드내용'),
            'job_status_codes': ('코드정보', 'jobCd', '코드', '코드내용'),
            'education_level_codes': ('코드정보', 'schoolCd', '코드', '코드내용'),
            'specialization_codes': ('코드정보', 'sBizCd', '코드', '코드내용'),
            'category_codes': ('정책대분류', None, '번호', '정책대분류명(lclsfNm)'),
            'subcategory_codes': ('정책중분류', None, '번호', '정책중분류명(mclsfNm)'),
            'keywords': ('정책키워드', None, None, '정책키워드명(plcyKywdNm)')
        }
        
        for table_name, (sheet_name, filter_col, code_col, name_col) in code_sheets_map.items():
            print(f"  - 테이블 '{table_name}' 작업 중 (시트: '{sheet_name}')...")
            # read_excel을 사용하여 특정 시트를 읽어옵니다.
            df = pd.read_excel(excel_file_path, sheet_name=sheet_name)
            
            # '코드정보' 시트의 경우, 특정 분류(jobCd 등)로 필터링
            if filter_col:
                df = df[df['분류'] == filter_col].copy()
            
            insert_sql = ""
            # 각 테이블 구조에 맞게 데이터 준비 및 INSERT
            if table_name == 'keywords':
                insert_sql = f"INSERT INTO {table_name} (name) VALUES (%s)"
                data_to_insert = [(row[name_col],) for index, row in df.iterrows() if pd.notna(row[name_col])]
            else:
                insert_sql = f"INSERT INTO {table_name} (code, name) VALUES (%s, %s)"
                data_to_insert = [(str(row[code_col]), row[name_col]) for index, row in df.iterrows() if pd.notna(row[code_col]) and pd.notna(row[name_col])]
            
            cursor.executemany(insert_sql, data_to_insert)
            print(f"    ✅ {cursor.rowcount}개 행 삽입 완료.")

    except FileNotFoundError:
        print(f"🚨 오류: 엑셀 파일('{excel_file_path}')을 찾을 수 없습니다. 파일 이름과 경로를 확인하세요.")
        raise
    except Exception as e:
        print(f"🚨 오류: 코드 테이블 이관 중 문제 발생 - {e}")
        raise


In [10]:
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': '1234',
    'database': 'toyprj4'
}

# 엑셀 파일 이름 (사용자님이 변경하신 이름)
excel_file_path = './code_table.xlsx'

In [14]:
connection = None
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()
print("✅ 데이터베이스 연결 성공!")

populate_code_tables(cursor)

✅ 데이터베이스 연결 성공!
🚀 코드 테이블 데이터 이관 시작...
  - 테이블 'major_codes' 작업 중 (시트: '코드정보')...
    ✅ 1개 행 삽입 완료.
  - 테이블 'job_status_codes' 작업 중 (시트: '코드정보')...
    ✅ 1개 행 삽입 완료.
  - 테이블 'education_level_codes' 작업 중 (시트: '코드정보')...
    ✅ 1개 행 삽입 완료.
  - 테이블 'specialization_codes' 작업 중 (시트: '코드정보')...
    ✅ 1개 행 삽입 완료.
  - 테이블 'category_codes' 작업 중 (시트: '정책대분류')...
    ✅ 5개 행 삽입 완료.
  - 테이블 'subcategory_codes' 작업 중 (시트: '정책중분류')...
    ✅ 17개 행 삽입 완료.
  - 테이블 'keywords' 작업 중 (시트: '정책키워드')...
    ✅ 17개 행 삽입 완료.


In [16]:
connection.commit()
print("\n🎉 모든 작업이 성공적으로 완료되었습니다.")


🎉 모든 작업이 성공적으로 완료되었습니다.


In [13]:
# connection.rollback()